simulate the two player farm game

first, set up big config dictionary that specifies the game to be played

jan 2023


In [8]:
# imports
from random import randint
import numpy as np
import random
import datetime # for limiting calculation to wall clock time
import math
import copy
import matplotlib.pyplot as plt
import csv
import sys
import pandas as pd

import farmgame
from mcts import MCTS
from agents import RandomPolicy


In [9]:
# practice randomly selecting an action from the legal list and then taking it
TheFarm = farmgame.configure_game(layer="Items00",resourceCond="even",costCond="low",visibilityCond="full",redFirst=True)
state = TheFarm

done=False

while not done:
# for i in range(15):
    currentplayer = state.players[state.turn]
    print(currentplayer['name'] + "'s turn!")
    # print("current backpack: " + str(list(v['name'] for v in currentplayer['backpack']['contents'])))
    # print("current farm items: " + str(list(v['name'] for v in state.items)))
    # print("current farm items: " + str(list(v['status'] for v in state.items)))
    actions = state.legal_actions()
    if actions==[]:
        action=None
        print(currentplayer['name'] + " passes turn. ")
    else:
        print(list(a.id for a in actions))
        action = random.choice(actions) #actions[0]
        print(currentplayer['name'] + " player picks " + action.name)
    state = state.take_action(action,inplace=True) #pick first veg in list
    rwd, done = state.reward(currentplayer['name'])

print("Red Reward: ",state.reward('red'))
print("Purp Reward: ",state.reward('purple'))
    


# action=random.choice(actions)
# print(action.name)
# new_state = state.take_action(action)
# rwd, done = new_state.reward()
# print(rwd, done)
# new_state

red's turn!
['Tomato00', 'Turnip01', 'Turnip00', 'Strawberry01', 'Strawberry00', 'Eggplant00', 'Tomato01', 'Turnip02', 'redpillow']
red player picks turnip
purple's turn!
['Tomato00', 'Turnip01', 'Turnip00', 'Strawberry01', 'Strawberry00', 'Eggplant00', 'Tomato01', 'purplepillow']
purple player picks tomato
red's turn!
['Turnip01', 'Turnip00', 'Strawberry01', 'Strawberry00', 'Eggplant00', 'Tomato01', 'box', 'redpillow']
red player picks turnip
purple's turn!
['Turnip00', 'Strawberry01', 'Strawberry00', 'Eggplant00', 'Tomato01', 'box', 'purplepillow']
purple player picks turnip
red's turn!
['Strawberry01', 'Strawberry00', 'Eggplant00', 'Tomato01', 'box', 'redpillow']
red player picks box
purple's turn!
['Strawberry01', 'Strawberry00', 'Eggplant00', 'Tomato01', 'box', 'purplepillow']
purple player picks eggplant
red's turn!
['Strawberry01', 'Strawberry00', 'Tomato01', 'redpillow']
red player picks strawberry
purple's turn!
['Strawberry00', 'Tomato01', 'box', 'purplepillow']
purple player

In [3]:
# Creating a game from start - prototype for simulations w/ helping data output
objectLayer = ["Items00","Items01"] 
resourceCond = ["even"]
costCond = ["low"] 
visibilityCond = ["full"]
redFirst = [True]

rC = resourceCond[0]
cC = costCond[0]
vC = visibilityCond[0]
rF = redFirst[0]

oL = objectLayer[0]
gameNum = 0
game_df = {"gameNum": [], "player": [], "action": [], "veg_color": [], "steps": [], "helpful": [], "redRwd": [], "purpRwd": []}
for oL in objectLayer:
    farm_config = farmgame.configure_game(layer=oL,resourceCond=rC,costCond=cC,visibilityCond=vC,redFirst=rF)
    done = False
    
    # red
    red_agent = MCTS(time=2., C=2, max_moves = 10, color="red")
    # red_agent = RandomPolicy(game)
    # purple
    purp_agent = MCTS(time=2., C=2, max_moves = 10, color="purple")
    # purp_agent = RandomPolicy(game)
    
    state = farm_config
    # print("current farm items: " + str(list(v.name for v in state.items if v.status=="farm")))
    # print("current box: " + str(list(v.name for v in state.farmbox.contents)))
    t = 0
    while not done:
    
        red_agent.update(state) 
        purp_agent.update(state)
    
        currentplayer = state.players[state.turn]
        #print(currentplayer['name'] + "'s turn!")
        #print("current backpack: " + str(list(v.name for v in currentplayer['backpack']['contents'])))
        #print("current farm items: " + str(list(v.name for v in state.items)))
        #print("current box: " + str(list(v.name for v in state.farmbox.contents)))
    
        if currentplayer['name'] == "red":
            action = red_agent.choose_action()
        else:
            action = purp_agent.choose_action()
    
        #find out if action is helpful or not
        transition = farmgame.Transition(state,action)
        helpful = transition.is_helping()
    
        #get steps taken
        steps = state.get_steps(action)
    
        if action is None:
            print("THIS SHOULdn't happen")
            #print(currentplayer['name'] + " has no more moves.")
        else:
            #print(currentplayer['name'] + " player picks " + action.name)
            print("turn %s complete"%(t))
        state = state.take_action(action,inplace=True) #pick first veg in list
        rwd, done = state.reward(currentplayer['name'])
        t += 1
        #add action data to dictionary
        game_df["gameNum"].append(gameNum)
        game_df["player"].append(currentplayer['name'])
        game_df["action"].append(action.name) 
        game_df["veg_color"].append(action.color) 
        game_df["steps"].append(steps)
        game_df["helpful"].append(helpful)

    game_df["redRwd"] += ([state.reward('red')[0]] * t)
    game_df["purpRwd"] += ([state.reward('purple')[0]] * t)

    print("game %s done"%(gameNum))
    gameNum += 1

    #print("Red Reward: ",state.reward('red')[0])
    #print("Purp Reward: ",state.reward('purple')[0])


turn 0 complete
turn 1 complete
turn 2 complete
turn 3 complete
turn 4 complete
turn 5 complete
turn 6 complete
turn 7 complete
turn 8 complete
turn 9 complete
turn 10 complete
turn 11 complete
game 0 done
turn 0 complete
turn 1 complete
turn 2 complete
turn 3 complete
turn 4 complete
turn 5 complete
turn 6 complete
turn 7 complete
turn 8 complete
turn 9 complete
game 1 done


In [4]:
def aggregate_game_outputs(game_df):
    helping_event = game_df.helpful.sum()
    steps = game_df.steps.sum()
    helpful_steps = game_df.loc[game_df.helpful,"steps"].sum()
    helped = helping_event > 0 
    purpleHelpingEvents = game_df.loc[game_df.player == "purple","helpful"].sum()
    redHelpingEvents = game_df.loc[game_df.player == "red","helpful"].sum()
    purpleHelped = purpleHelpingEvents > 0 
    redHelped = redHelpingEvents > 0 
    purpleHelpfulSteps = game_df.loc[(game_df.player == "purple") & (game_df.helpful),"steps"].sum()
    redHelpfulSteps = game_df.loc[(game_df.player == "red") & (game_df.helpful),"steps"].sum()
    nRedVeg = (game_df.veg_color == "red").sum()
    nPurpVeg = (game_df.veg_color == "purple").sum()
    
    agg_mets = pd.Series({"helping_event":helping_event, "steps": steps, "helpful_steps": helpful_steps, "helped": helped, "purpleHelpingEvents": purpleHelpingEvents,
                          "redHelpingEvents": redHelpingEvents, "purpleHelped": purpleHelped, "redHelped": redHelped, "purpleHelpfulSteps": purpleHelpfulSteps, 
                          "redHelpfulSteps": redHelpfulSteps, "nRedVeg": nRedVeg, "nPurpVeg": nPurpVeg}) 
    return agg_mets


In [5]:
game_df = pd.DataFrame(game_df)
game_df_results = game_df.groupby("gameNum").apply(aggregate_game_outputs)
game_df_results

/var/folders/6r/gppxzrp57j9ggbttd9chp8cm0000gn/T/ipykernel_23408/4244742291.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  game_df_results = game_df.groupby("gameNum").apply(aggregate_game_outputs)


,helping_event,steps,helpful_steps,helped,purpleHelpingEvents,redHelpingEvents,purpleHelped,redHelped,purpleHelpfulSteps,redHelpfulSteps,nRedVeg,nPurpVeg
gameNum,,,,,,,,,,,,
0,2,106,24,True,1,1,True,True,13,11,5,5
1,4,75,24,True,2,2,True,True,13,11,4,4


In [6]:
# compare random - random, mcts - random, random - mcts, mcts-mcts

games = [{"red_agent": RandomPolicy(),"purp_agent": RandomPolicy()},
        {"red_agent": MCTS(time=5., C=2, max_moves = 10),"purp_agent": RandomPolicy()},
        {"red_agent": RandomPolicy(),"purp_agent": MCTS(time=5., C=2, max_moves = 10)},
        {"red_agent": MCTS(time=5., C=2, max_moves = 10),"purp_agent": MCTS(time=5., C=2, max_moves = 10)},]
rwds = []

for i in range(len(games)):
    print("GAME "+str(i))
    action_seq = []
    state = farmgame.configure_game() # DEFAULT GAME
    done = False

    red_agent = games[i]["red_agent"]
    purp_agent = games[i]["purp_agent"]
    
    while not done:

        red_agent.update(state) 
        purp_agent.update(state)

        currentplayer = state.players[state.turn]
        # print(currentplayer['name'] + "'s turn!")
        # print("current backpack: " + str(list(v['name'] for v in currentplayer['backpack']['contents'])))
        # print("current farm items: " + str(list(v['name'] for v in state.items)))
        # print("current box: " + str(list(v['name'] for v in state.farmbox['contents'])))

        if currentplayer['name'] == "red":
            action = red_agent.choose_action()
        else:
            action = purp_agent.choose_action()

        # actions = state.legal_actions(currentplayer)
        # print(list(a.name for a in actions))
        # action = actions[0]
        transition = farmgame.Transition(state,action)
        helpful = transition.is_helping()
        if action is None:
            print("THIS SHOULdn't happen")
            print(currentplayer['name'] + " has no more moves.")
        else:
            print(currentplayer['name'] + " player picks " + action.name, ", helpful: %s"%(helpful))

        state = state.take_action(action,inplace=True) #pick first veg in list
        rwd, done = state.reward(currentplayer['name'])

    print("Red Reward: ",state.reward('red')[0])
    print("Purp Reward: ",state.reward('purple')[0])
    rwds.append({"red":state.reward('red')[0],"purple":state.reward('purple')[0]})
    

GAME 0
red player picks strawberry , helpful: False
purple player picks turnip , helpful: False
red player picks eggplant , helpful: True
purple player picks tomato , helpful: True
red player picks turnip , helpful: True
purple player picks tomato , helpful: True
red player picks strawberry , helpful: False
purple player picks pillow , helpful: False
red player picks pillow , helpful: False
purple player picks turnip , helpful: False
red player picks pillow , helpful: False
purple player picks pillow , helpful: False
red player picks box , helpful: False
purple player picks pillow , helpful: False
red player picks none , helpful: False
purple player picks pillow , helpful: False
red player picks none , helpful: False
purple player picks box , helpful: False
Red Reward:  156
Purp Reward:  128
GAME 1
red player picks strawberry , helpful: False
purple player picks tomato , helpful: True
red player picks turnip , helpful: True
purple player picks eggplant , helpful: False
red player picks

In [7]:

# now let's compare the performance of different C

storestates = []
storeactions = []
storerewards_red = []
storerewards_purp = []
cs = [8,10,25,50,100,150]

n_reps = 10
all_rwds_c_r = []
all_rwds_c_p = []

for c in cs:
    print("C = ", c)
    storerewards_red = []
    storerewards_purp = []

    for i in range(n_reps):
    
        state = farmgame.configure_game(layer="Items00",resourceCond="even",costCond="low",visibilityCond="full",redFirst=True)
        done = False

        # red
        red_agent = MCTS(time=5., C=c, max_moves = 10, color="red") #, verbose=True)

        # purple
        purp_agent = MCTS(time=5., C=c, max_moves = 10, color="purple")

        action_seq = []


        while not done:

            red_agent.update(state) 
            purp_agent.update(state)

            currentplayer = state.players[state.turn]

            if currentplayer['name'] == "red":
                action = red_agent.choose_action()
            else:
                action = purp_agent.choose_action()

            # print(currentplayer['name'] + " player picks " + action['name'])
            
            state = state.take_action(action,inplace=True) #pick first veg in list
            rwd, done = state.reward(currentplayer['name'])

            action_seq.append(action)


        print("Red Reward: ",state.reward('red')[0])
        print("Purp Reward: ",state.reward('purple')[0])
        
        storestates.append([red_agent.states])
        storeactions.append([action_seq])
        storerewards_red.append(state.reward('red')[0])
        storerewards_purp.append(state.reward('purple')[0])

    all_rwds_c_r.append(storerewards_red)
    all_rwds_c_p.append(storerewards_purp)


C =  8
Red Reward:  276
Purp Reward:  288
Red Reward:  220
Purp Reward:  192
Red Reward:  236
Purp Reward:  200
Red Reward:  168
Purp Reward:  160
Red Reward:  204
Purp Reward:  248
Red Reward:  276
Purp Reward:  296
Red Reward:  236
Purp Reward:  184
Red Reward:  212
Purp Reward:  240
Red Reward:  228
Purp Reward:  248
Red Reward:  164
Purp Reward:  200
C =  10
Red Reward:  228
Purp Reward:  160
Red Reward:  244
Purp Reward:  248
Red Reward:  176
Purp Reward:  196
Red Reward:  164
Purp Reward:  160
Red Reward:  188
Purp Reward:  204
Red Reward:  228
Purp Reward:  228
Red Reward:  228
Purp Reward:  176
Red Reward:  204
Purp Reward:  248
Red Reward:  244
Purp Reward:  208
Red Reward:  236
Purp Reward:  200
C =  25
Red Reward:  204
Purp Reward:  240
Red Reward:  156
Purp Reward:  208
Red Reward:  148
Purp Reward:  220
Red Reward:  148
Purp Reward:  216
Red Reward:  236
Purp Reward:  200
Red Reward:  164
Purp Reward:  160
Red Reward:  196
Purp Reward:  176
Red Reward:  204
Purp Reward:  1

KeyboardInterrupt: 

In [ ]:
# cashmoney_r = [i for (i,a) in storerewards_red]
# cashmoney_p = [i for (i,a) in storerewards_purp]

# plt.plot(cs,storerewards_red,'o-',color="red")
# plt.plot(cs,storerewards_purp,'o-',color="purple")

plt.plot(cs,np.mean(all_rwds_c_r),yerr=np.std(all_rwds_c_r),color="red")
plt.plot(cs,np.mean(all_rwds_c_p),yerr=np.std(all_rwds_c_p),color="purple")
plt.xlabel("C")
plt.ylabel("reward")
plt.show()

In [ ]:
# now let's compare the performance of different computation times

storetstates = []
storetactions = []

# 1min, 2min, 3min, 4min, 5min, 6min per game approx
times = [0.1,0.5,1,2,5,10,15,20,25,30]

# set c as desired
c=100

# 10*(1 + 2 + 3 + 4 + 5 + 6) = 210 min approx (~3.5 hours)
n_reps = 10
all_rwds_r = []
all_rwds_p = []

for t in times:
    print("Computation time = ", t)
    storetrewards_red = []
    storetrewards_purp = []

    for i in range(n_reps):

        # Initialize the tamagotchi
        state = farmgame.configure_game(layer="Items00",resourceCond="even",costCond="low",visibilityCond="full",redFirst=True)
        done = False

         # red
        red_agent = MCTS(time=t, C=c, max_moves = 10, color="red")

        # purple
        purp_agent = MCTS(time=t, C=c, max_moves = 10, color="purple")

        action_seq = []
        
        while not done:
            red_agent.update(state) 
            purp_agent.update(state)

            currentplayer = state.players[state.turn]

            if currentplayer['name'] == "red":
                action = red_agent.choose_action()
            else:
                action = purp_agent.choose_action()
            
            state = state.take_action(action,inplace=True) #pick first veg in list
            rwd, done = state.reward(currentplayer['name'])

            action_seq.append(action)

        print("Red Reward: ",state.reward('red')[0])
        print("Purp Reward: ",state.reward('purple')[0])
        
        storetstates.append([red_agent.states])
        storetactions.append([action_seq])
        storetrewards_red.append(state.reward('red')[0])
        storetrewards_purp.append(state.reward('purple')[0])

    all_rwds_r.append(storetrewards_red)
    all_rwds_p.append(storetrewards_purp)

In [ ]:
print(all_rwds_r)
print(np.shape(all_rwds_r))
np.mean(all_rwds_r,axis=1)

In [ ]:
# plot rwd vs comp time

plt.errorbar(times,np.mean(all_rwds_r,axis=1),yerr=np.std(all_rwds_r),color="red")
plt.errorbar(times,np.mean(all_rwds_p,axis=1),yerr=np.std(all_rwds_p),color="purple")
plt.xlabel("Computation time limit (s)")
plt.ylabel("Reward")
plt.xscale('log')
plt.show()

In [ ]:
storelstates = []
storelactions = []
storelrewards_red = []
storelrewards_purp = []
layers = ["Items00","Items01","Items02","Items03","Items04","Items05","Items06","Items07","Items08","Items09","Items10","Items11"]

for l in layers:
    print("Layer = ", l)
    
    # Initialize the tamagotchi
    state = farmgame.configure_game(layer = l)
    game = farmgame.FarmGame()
    done = False

    # red
    red_agent = MCTS(game, time=2., C=1, max_moves = 20)

    # purple
    purp_agent = MCTS(game, time=2., C=1, max_moves = 20)

    action_seq = []
    
    while not done:
        red_agent.update(state) 
        purp_agent.update(state)

        currentplayer = state.players[state.turn]
        # print("*** " + currentplayer.name + "'s turn! ***")
        # print("current farm items: " + str(list(v.name for v in state.items if v.status=="farm")))
        # print("current backpack: " + str(list(v.name for v in currentplayer.backpack.contents)))
        # print("current box: " + str(list(v.name for v in state.farmbox.contents)))

        if currentplayer.name == "red":
            action = red_agent.choose_action()
        else:
            action = purp_agent.choose_action()

        action_seq.append(action)

        #find out if chosen action is helpful
        transition = farmgame.Transition(state,action)
        helpful = transition.is_helping()

        #get number of steps
        steps = state.get_steps(action)
        print(helpful, steps)

        if action is None:
            print(currentplayer.name + " has no more moves.")
        else:
            print(currentplayer.name + " player picks " + action.name)
        state = state.take_action(action,inplace=True) #pick first veg in list
        rwd, done = state.reward(currentplayer.name)

    print("Red Reward: ",game.reward(red_agent.states,'red'))
    print("Purp Reward: ",game.reward(purp_agent.states,'purple'))
    
    storelstates.append([red_agent.states])
    storelactions.append([action_seq])
    storelrewards_red.append(game.reward(red_agent.states,'red'))
    storelrewards_purp.append(game.reward(purp_agent.states,'purple'))

In [ ]:
print(storelrewards_red)
print(storelrewards_purp)